In [ ]:
from allennlp.predictors.predictor import Predictor
import allennlp_models.tagging
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_sm')


predictor = Predictor.from_path('https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz')

In [ ]:
allen_text = 'It is hard not to get excited about a country of 1.2 billion people whose economy grew by more than 12% in real terms last year and could well grow by a further 10% or so in 1993. But excited is a mild word for what foreign investors now feel about China. The Chinese government says that last year foreigners committed themselves to invest $58 billion in 40,000 projects in China. In the end this figure will prove to have been an exaggeration of China-like proportions, but the $9 billion-10 billion-worth of foreign investment that actually made its way into the country last year was still almost the same as was by the United States, the world\'s biggest economy. Foreign traders, investors and buyers of securities are right, in the long run, to be devoting a lot of attention to China. Last year the dollar value of China\'s exports rose 22%, to $85 billion, and imports 27%, to $81 billion; this year China\'s trade should exceed $200 billion, making it roughly the world\'s tenth-biggest trading nation. China is already the world\'s biggest importer of wheat. It will probably be the biggest telecoms-equipment market between now and 2000. And a swathe of 300m people in China\'s coastal provinces are about to experience a huge consumer-goods boom (see page 70).Nonetheless, many foreigners, abetted by overly cheerful Chinese economists, are tempted to forget that China is no more exempt from the laws of economic gravity than was Japan in the late 1980s, when some people claimed that the Rising Sun and its stockmarket would go on rising for ever. Even if a trade war between China and Bill Clinton\'s America is averted, this year and next are liable to be awkward for China, with crashing stockmarkets and property prices and a clampdown on credit. Not only will China itself be afflicted; so will places whose economies have become ever more entwined with China\'s, in particular Hong Kong.AFTER GO, STOPThe main reason to expect trouble in China is inflation. Ever since Deng Xiaoping\'s economic reforms began in the late 1970s, the liberalising Chinese economy has been plagued by stop-go cycles of increasing severity. Two previous rounds of double-digit economic growth that got out of hand-in 1984-85 and 1987-88-led to frightening price rises, unbridled corruption and (indirectly, in the second instance) the Tiwananmen Square killing of June 1989. Optimists in China and Hong Kong say that history is not about to repeat itself. First, they note, even after two years of economic expansion there is little sign of heavy pressure on prices. Consumer prices nationwide rose by less than 6% in 1992, though by twice that in the big cities. Second, China is no longer an economy of shortages. In 1988 people began hoarding goods in panic because they interpreted the initial rise in prices as a forerunner of the disappearance of the goods. Today, with most goods distributed through the market rather than the plan, shop shelves are groaning and household bank deposits rising: no worries here about vanishing goods or a collapsing currency. Third, China\'s leaders are acutely aware of the disaster that befell the country in 1988-89 when they dawdled about cooling the economy. They will not, runs the argument, make that mistake again.All this is unconvincing. It is true that, with extensive price reform and freeish foreign trade and investment, China has escaped from the chronic shortages typical of communism. But so have capitalist countries, and that has hardly immunised them from inflation.China\'s price rises have so far been modest, but the warning bells are clanging. The narrow measure of money, MO, may have risen by as much as 40% last year; M2 grew by almost 30% a year between mid-1990 and mid-1992, and for 1992 as a whole it probably grew faster than 30%. Prices of raw materials rose sharply last year, property prices in the coastal boom-towns are soaring, the spot-market exchange rate for the yuan has fallen far, and the current account is deteriorating. Worst of all, even if China\'s leaders are reading these sins correctly, they have no monetary instruments more discriminating than credit quotas to use in trying to forestall a burst of price rises.A stop-go lurch is in fact built into China\'s still only half-reformed economy. The bright side to the upcoming price troubles is that they might at last create the conditions for a break-through to true capitalism in China (see page 61). But meanwhile the risks of instability are high. For company presidents and the American president alike, this is the time for a long view rather than a quick killing in China.'


pred_obj = predictor.predict(
    document = allen_text
)

In [27]:
spacy_df = pd.DataFrame([],columns=['text', 'lemma', 'pos', 'tag', 'dep'])

document_clusters = pred_obj['clusters']
document_list = pred_obj['document']
spacy_doc = nlp(allen_text)

name = ['Bill', 'Clinton']

for token in spacy_doc:
    tok_observation = [token.text, token.lemma_, token.pos_, token.tag_, token.dep_]
    spacy_df.loc[len(spacy_df)] = tok_observation

# print(spacy_doc)


cluster_of_interest = identifyMainCluster(document_clusters, name, document_list)

replaced = replaceCoref(name, document_clusters, cluster_of_interest, document_list, spacy_df)

Bill Clinton
Bill Clinton's
last year
last year
last year
Last year
last year
last year
foreigners
themselves
China
China
the country
China
China 's
China 's
it
China
It
China 's
China
China
China
China itself
itself
China 's
China
China
China
China 's
the country
China
China 's
China 's
China 's
China
China
$ 58 billion
this figure
the world 's
the world 's
the world 's
Japan
its
this year
this year
1992
1992 as a whole
Hong Kong
Hong Kong
history
itself
Optimists in China and Hong Kong
they
people
they
goods
the goods
1988
1988 - 89
China 's leaders
they
They
China 's leaders
they
dawdled
that mistake
capitalist countries
them
M2
it
the liberalising Chinese economy
China 's still only half - reformed economy
the upcoming price troubles
they


TypeError: list indices must be integers or slices, not NoneType

In [ ]:
# spacy_list = list(tok.text_with_ws for tok in spacy_doc)
# print(spacy_list)
# print(''.join(spacy_list))
# print(replaced)
print(' '.join(replaced))

In [26]:
def identifyMainCluster(clusters, name_vect, document):
    clusters = clusters
    name = ' '.join(name_vect)

    print(name)
    print(name+"\'s")
    # print(clusters)

    for cluster in clusters:
        for index_pair in cluster:
            start_index = index_pair[0]
            stop_index = index_pair[1]+1
            cluster_as_string = ' '.join(document[start_index:stop_index])

            print(cluster_as_string)
            if (cluster_as_string == name) or (cluster_as_string == name+"\'s"):
                print('here')
                cluster_index = clusters.index(cluster)
                return cluster_index

In [ ]:
def replaceCoref(name, clusters, cluster_interest_index, document, tokenized_df):
    name_pos = name[:]
    name_pos[1] = name_pos[1] + "\'s"

    base_cluster = clusters[cluster_interest_index]
    rev_cluster  = sorted(base_cluster, key = lambda x: x[0], reverse = True)

    new_doc_list = document

    df = tokenized_df
    for index_pair in rev_cluster:
        start_index = index_pair[0]
        stop_index = index_pair[1]+1

        # Replacing one element (i.e. pronouns/possessives)
        if start_index == stop_index:
            if containsPossessive(df, start_index, stop_index) == True:
                new_doc_list[start_index:stop_index] = name_pos
            else:
                new_doc_list[start_index:stop_index] = name
            new_doc_list.pop(start_index+2)
        # Replacing two or more elements
        else:
            if containsPossessive(df, start_index, stop_index) == True:
                new_doc_list[start_index:stop_index] = name_pos
            else:
                new_doc_list[start_index:stop_index] = name

    
    return new_doc_list

In [ ]:
def containsPossessive(df, start_index, stop_index):
    possessive_tags = ['POS', 'PRP$']
    cluster_tags = df['tag'].iloc[start_index:stop_index]
    print(start_index, stop_index)
    print(df['text'].iloc[start_index:stop_index])
    print(set(possessive_tags))
    # .isdisjoint returns true if the two sets are disjoin (i.e. do not have any elements interesecting (i.e. does not contain a POS or PRP$)). So negate to return True if "NOT DISJOINT" (CONTAINS POS/PRP$)
    if not set(possessive_tags).isdisjoint(set(cluster_tags)):
        return True
    else:
        return False

In [ ]:
clusters = pred_obj['clusters']
document = pred_obj['document']

## first identify the indices of cluster of interest (i.e. the name)
def identifyMainCluster(name_vect, pred_obj):
    clusters = pred_obj['clusters']
    
    name = ' '.join(name_vect)

    for cluster in clusters:
        for index_pair in cluster:
            start_index = index_pair[0]
            stop_index = index_pair[1]+1
            as_string = ' '.join(document[start_index:stop_index])
            print(as_string)

            if as_string == name:
                cluster_index = clusters.index(cluster)
                name_index = cluster.index(index_pair)
                return cluster_index, name_index

def replaceCoref(document, pred_obj):
    name_vect = ['Paul', 'Allen']
    cluster_of_interest, pair_of_interest = identifyMainCluster(name_vect, pred_obj)

    cluster = pred_obj['clusters'][cluster_of_interest]

    cluster_rev = sorted(cluster, key = lambda x: x[0], reverse = True)
    document = pred_obj['document']
    for pair in cluster_rev:
        start = pair[0]
        stop = pair[1]

        # Replacing one element (i.e. pronouns)
        if start==stop:
            document[start:stop] = name_vect
            document.pop(start+2)

        # Replacing two elements (cleanest)
        elif start == stop-1:
            document[start:stop] = name_vect
        # Replacing long strings
        else:
            document[start:stop] = name_vect

        
        





In [ ]:
test_doc = ['a','b','c','d','e','f','g','h','i','j']

replacement_string = ['REPLACER1', 'REPLACER2']

test_index1 = [0,1]
test_index2 = [3,3]
test_index3 =[5,8]

cluster = [test_index1, test_index3, test_index2]
rev_cluster = sorted(cluster, key=lambda x: x[0], reverse=True)
print(rev_cluster)


for pair in rev_cluster:
    start = pair[0]
    stop = pair[1]

    # 1-word replacement (pronouns)
    if start==stop:
        test_doc[start:stop] = replacement_string
        print(start+2)
        test_doc.pop(start+2)

    # 2-word replacement (honorary plus last name)
    elif start==stop-1:
        test_doc[start:stop] = replacement_string 
    # multi-word replacement
    else:
        test_doc[start:stop] = replacement_string


print(test_doc)

In [ ]:
test = predictor.coref_resolved(
    document = "Paul Allen was born on January 21, 1953, in Seattle, Washington, to Kenneth Sam Allen and Edna Faye Allen. Allen attended Lakeside School, a private school in Seattle, where he befriended Bill Gates, two years younger, with whom he shared an enthusiasm for computers. Paul and Bill used a teletype terminal at their high school, Lakeside, to develop their programming skills on several time-sharing computer systems. His yacht, 'Octopus' cost 0.876 billion dollars."
)

print(test)

In [31]:
for i in range(10):
    if i==6:
        continue
    print(i)

0
1
2
3
4
5
7
8
9
